<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/12_%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 12_卷积神经网络

### 卷积网络层

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

类风格实现（nn.Xxx）：要先实例化再调用

In [0]:
x=torch.rand(1,1,28,28) #核个数，输入通道数，核大小
layer=nn.Conv2d(1,3,kernel_size=3,stride=1,padding=0) #卷积层，类风格实现：输入通道数，核数量

out=layer.forward(x) #卷积前向运算（不建议用）
out.shape

torch.Size([1, 3, 26, 26])

In [0]:
out=layer(x) #卷积前向运算，用封装好的方法，直接是实例+()
out.shape

torch.Size([1, 3, 26, 26])

In [0]:
layer.weight #查看网络层里的参数，[3,1,3,3]核数量 输入通道 核大小，w b会自动跟新

Parameter containing:
tensor([[[[-0.3300,  0.0198, -0.1243],
          [ 0.0597, -0.0743,  0.2338],
          [-0.2747,  0.2101,  0.2337]]],


        [[[-0.2405,  0.1064,  0.0402],
          [ 0.3295,  0.1337,  0.0645],
          [ 0.0190, -0.0427,  0.1558]]],


        [[[-0.3241,  0.0522, -0.0070],
          [ 0.0812, -0.2457, -0.1421],
          [ 0.1151,  0.0057, -0.1088]]]], requires_grad=True)

函数风格实现（F.xxx）：自己管理这里tensor、运算过程，仅仅用了它GUP加速的过程，不用实例化。

pytorch默认大写是类，小写是接口

In [0]:
w=torch.rand(16,3,5,5) #核个数，输入通道数，核大小
b=torch.rand(16)
x=torch.randn(1,3,28,28) #核个数，输入通道数，核大小

out=F.conv2d(x,w,b,stride=1,padding=1) #CNN的一层，底层函数风格实现
out.shape

torch.Size([1, 16, 26, 26])

### 上下采样

池化

In [0]:
x=torch.rand(1,16,14,14)

In [0]:
layer=nn.MaxPool2d(2,stride=2) #最大值池化，类风格实现：窗口大小，步长
out=layer(x)
out.shape #减半了

torch.Size([1, 16, 7, 7])

In [0]:
layer=F.avg_pool2d(x,2,stride=2) #最大值池化，函数风格实现
out.shape

torch.Size([1, 16, 7, 7])

上采样

In [0]:
out=F.interpolate(x,scale_factor=3,mode='nearest') #插值法上采样：tensor，放大倍数，插值方法（近邻、双性、三性）
out.shape #

torch.Size([1, 16, 42, 42])

### ReLU函数

In [0]:
x.shape

torch.Size([1, 16, 14, 14])

In [0]:
layer=nn.ReLU(inplace=True) #ReLU函数，维度不变，inplace=True就是让结果替换原来的x的地址，节省内存
out=layer(x)
out.shape

torch.Size([1, 16, 14, 14])

In [0]:
layer=F.relu(x)
out.shape

torch.Size([1, 16, 14, 14])

### Batch Normalization

一维

In [0]:
x=torch.rand(100,16,784) #均匀分布的均值是0.5，方差为1
layer=nn.BatchNorm1d(16) #Batch Normal;用1d是因为1维的feature 28*28，16就是通道数
out=layer(x) #统计出当前x的u和o^2
out.shape

torch.Size([100, 16, 784])

In [0]:
layer.running_mean #更新全局的running_u，当只有一次运算时就是u，都在0.5附近

tensor([0.0500, 0.0499, 0.0498, 0.0500, 0.0500, 0.0499, 0.0497, 0.0499, 0.0498,
        0.0500, 0.0500, 0.0498, 0.0499, 0.0498, 0.0499, 0.0500])

In [0]:
layer.running_var #更新全局的running_o^2，方差为1左右

tensor([0.9083, 0.9084, 0.9083, 0.9083, 0.9083, 0.9083, 0.9084, 0.9083, 0.9083,
        0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9084, 0.9083])

二维

In [0]:
x=torch.rand(1,16,7,7)
x.shape

torch.Size([1, 16, 7, 7])

In [0]:
layer=nn.BatchNorm2d(16)
out=layer(x)
out.shape #大小不变

torch.Size([1, 16, 7, 7])

In [0]:
layer.weight #相当于r，需要梯度

Parameter containing:
tensor([0.3847, 0.9785, 0.7068, 0.7617, 0.5684, 0.7968, 0.0674, 0.8322, 0.9962,
        0.2803, 0.3346, 0.1811, 0.4323, 0.6628, 0.6418, 0.6469],
       requires_grad=True)

In [0]:
layer.bias #相当于B

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)

In [0]:
vars(layer) #Class variables

{'_backend': <torch.nn.backends.thnn.THNNFunctionBackend at 0x7f5edd095668>,
 '_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict([('running_mean',
               tensor([0.0450, 0.0558, 0.0435, 0.0471, 0.0516, 0.0510, 0.0461, 0.0421, 0.0493,
                       0.0516, 0.0499, 0.0498, 0.0552, 0.0517, 0.0464, 0.0504])),
              ('running_var',
               tensor([0.9090, 0.9087, 0.9091, 0.9075, 0.9097, 0.9088, 0.9078, 0.9065, 0.9085,
                       0.9097, 0.9072, 0.9109, 0.9090, 0.9070, 0.9080, 0.9054])),
              ('num_batches_tracked', tensor(1))]),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_parameters': OrderedDict([('weight', Parameter containing:
               tensor([0.3847, 0.9785, 0.7068, 0.7617, 0.5684, 0.7968, 0.0674, 0.8322, 0.9962,
                       0.2803, 0.3346, 0.1811, 0.4323, 0.6628, 0.6418, 0.6469],
                      requ

测试里的Batch Norm

In [0]:
layer.eval()
nn.BatchNorm1d(16,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)
out=layer(x)
out.shape

torch.Size([1, 16, 7, 7])

### ResNet的残差单元

In [0]:
import  torch
from    torch import  nn
from    torch.nn import functional as F

# from    torchvision.models import resnet18

# 残差单元（适用于浅层网络，中间有两层卷积层）
class ResBlk(nn.Module):
    """
    resnet block
    """

    def __init__(self, ch_in, ch_out):
        """
        :param ch_in:
        :param ch_out:
        """
        super(ResBlk, self).__init__()

        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)

        self.extra = nn.Sequential()
        if ch_out != ch_in: # 确保x跳连的输出是ch_out，以便与卷积层像素相加
            # [b, ch_in, h, w] => [b, ch_out, h, w]
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out)
            )

    def forward(self, x):
        """
        :param x: [b, ch, h, w]
        :return:
        """
        out = F.relu(self.bn1(self.conv1(x))) # 这里加relu
        out = self.bn2(self.conv2(out))
        # short cut，extra module: [b, ch_in, h, w] => [b, ch_out, h, w]
        out = self.extra(x) + out # 元素相加（element-wise add）

        return out

### ResNet网络
ResNet18跑CIFAR10

In [0]:
import  torch
from    torch import  nn
from    torch.nn import functional as F
from    torch.utils.data import DataLoader
from    torchvision import datasets
from    torchvision import transforms
from    torch import nn, optim

# from    torchvision.models import resnet18

# 残差单元（适用于浅层网络，中间有两层卷积层）
class ResBlk(nn.Module):
    """
    resnet block
    """

    def __init__(self, ch_in, ch_out):
        """
        :param ch_in:
        :param ch_out:
        """
        super(ResBlk, self).__init__()

        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)

        self.extra = nn.Sequential()
        if ch_out != ch_in: # 确保x跳连的输出是ch_out，以便与卷积层像素相加
            # [b, ch_in, h, w] => [b, ch_out, h, w]
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out)
            )

    def forward(self, x):
        """
        :param x: [b, ch, h, w]
        :return:
        """
        out = F.relu(self.bn1(self.conv1(x))) # 这里加relu
        out = self.bn2(self.conv2(out))
        # short cut，extra module: [b, ch_in, h, w] => [b, ch_out, h, w]
        out = self.extra(x) + out # 元素相加（element-wise add）

        return out


# ResNet18网络结构
class ResNet18(nn.Module):

    def __init__(self):
        super(ResNet18, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16)
        )
        # followed 4 blocks
        # [b, 64, h, w] => [b, 128, h ,w]
        self.blk1 = ResBlk(16, 16)
        # [b, 128, h, w] => [b, 256, h, w]
        self.blk2 = ResBlk(16, 32)
        # # [b, 256, h, w] => [b, 512, h, w]
        # self.blk3 = ResBlk(128, 256)
        # # [b, 512, h, w] => [b, 1024, h, w]
        # self.blk4 = ResBlk(256, 512)

        self.outlayer = nn.Linear(32*32*32, 10)

    def forward(self, x):
        """
        :param x:
        :return:
        """
        x = F.relu(self.conv1(x))

        # [b, 64, h, w] => [b, 1024, h, w]
        x = self.blk1(x)
        x = self.blk2(x)
        # x = self.blk3(x)
        # x = self.blk4(x)

        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.outlayer(x)

        return x


def main():
    batchsz = 32

    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)


    x, label = iter(cifar_train).next()
    print('x:', x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    # model = Lenet5().to(device)
    model = ResNet18().to(device)

    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    print(model)

    for epoch in range(1000):

        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # [b, 3, 32, 32]
            # [b]
            x, label = x.to(device), label.to(device)

            logits = model(x)
            # logits: [b, 10]
            # label:  [b]
            # loss: tensor scalar
            loss = criteon(logits, label)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        #
        print(epoch, 'loss:', loss.item())


        model.eval()
        with torch.no_grad():
            # test
            total_correct = 0
            total_num = 0
            for x, label in cifar_test:
                # [b, 3, 32, 32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b, 10]
                logits = model(x)
                # [b]
                pred = logits.argmax(dim=1)
                # [b] vs [b] => scalar tensor
                correct = torch.eq(pred, label).float().sum().item()
                total_correct += correct
                total_num += x.size(0)
                # print(correct)

            acc = total_correct / total_num
            print(epoch, 'acc:', acc)


if __name__ == '__main__':
    main()

0it [00:00, ?it/s]

170500096it [00:06, 27783743.02it/s]                               


Files already downloaded and verified
x: torch.Size([32, 3, 32, 32]) label: torch.Size([32])
ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk1): ResBlk(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra): Sequential()
  )
  (blk2): ResBlk(
    (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, 